In [4]:
!pip install redis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.9/339.9 KB 850.8 kB/s eta 0:00:00a 0:00:01


In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import redis

In [15]:
# Конфигурация Valkey
valkey_config = {
    "host": "valkey",
    "port": 6379,
    "db": 0,
    "password": "bigdata2024",  # пароль из docker-compose
    "decode_responses": True
}

spark = SparkSession.builder \
    .appName("ValkeyETL") \
    .getOrCreate()

In [16]:
def test_valkey_connection():
    """Тестирование подключения к Valkey"""
    try:
        r = redis.Redis(**valkey_config)
        r.ping()
        print("✓ Valkey подключение успешно")
        return True
    except Exception as e:
        print(f"✗ Ошибка подключения к Valkey: {e}")
        return False

test_valkey_connection()

✓ Valkey подключение успешно


True

In [17]:
pg_url = "jdbc:postgresql://postgres:5432/bober_db"
pg_properties = {"user": "bober", "password": "bober", "driver": "org.postgresql.Driver"}
df = spark.read.jdbc(url=pg_url, table="mock_data", properties=pg_properties)

# Загружаем все таблицы звезды
fact = spark.read.jdbc(url=pg_url, table="fact_sales", properties=pg_properties)
dim_product = spark.read.jdbc(url=pg_url, table="dim_product", properties=pg_properties)
dim_customer = spark.read.jdbc(url=pg_url, table="dim_customer", properties=pg_properties)
dim_store = spark.read.jdbc(url=pg_url, table="dim_store", properties=pg_properties)
dim_supplier = spark.read.jdbc(url=pg_url, table="dim_supplier", properties=pg_properties)
dim_date = spark.read.jdbc(url=pg_url, table="dim_date", properties=pg_properties)
dim_date.head(1)

[Row(full_date=datetime.date(2021, 1, 1), date_id=1, year=2021, month=1, day=1, quarter=1)]

In [18]:
def save_dataframe_to_valkey(df, key_name, key_column="id"):
    """
    Сохраняет DataFrame в Valkey как Hash
    """
    df.write \
        .format("org.apache.spark.sql.redis") \
        .option("table", key_name) \
        .option("key.column", key_column) \
        .mode("overwrite") \
        .save()

def save_key_value_to_valkey(df, key_prefix, key_column, value_column):
    """
    Сохраняет пары ключ-значение в Valkey
    """
    df.select(
        concat(lit(key_prefix), col(key_column)).alias("key"),
        col(value_column).alias("value")
    ).write \
        .format("org.apache.spark.sql.redis") \
        .option("table", "temp") \
        .option("key.column", "key") \
        .mode("append") \
        .save()

In [19]:
def save_dataframe_to_valkey(df, key_prefix, id_column="id"):
    """
    Сохраняет DataFrame в Valkey используя RDD
    """
    def save_partition(partition):
        import redis
        
        # Создаем подключение для каждой партиции
        r = redis.Redis(**VALKEY_CONFIG)
        
        for row in partition:
            # Создаем ключ
            key = f"{key_prefix}:{row[id_column]}"
            
            # Сохраняем все поля как hash
            row_dict = row.asDict()
            for field, value in row_dict.items():
                if value is not None:
                    r.hset(key, field, str(value))
            
            print(f"Saved: {key}")
        
        r.close()
    
    # Сохраняем через RDD
    df.rdd.foreachPartition(save_partition)
    print(f"✅ Успешно сохранено в Valkey с префиксом: {key_prefix}")

def create_product_sales_dashboard(fact, dim_product):
    """
    Витрина 1: Продажи по продуктам
    """
    print("=== Создание витрины продаж по продуктам ===")
    
    # Топ-10 самых продаваемых продуктов
    top_products = fact.join(dim_product, "product_id") \
        .groupBy("product_id", "name", "category") \
        .agg(
            sum("sale_quantity").alias("total_quantity"),
            sum("sale_total_price").alias("total_revenue"),
            avg("rating").alias("avg_rating"),
            sum("reviews").alias("total_reviews")
        ) \
        .orderBy(desc("total_quantity")) \
        .limit(10)
    
    print("Топ-10 продуктов:")
    top_products.show()
    
    # Сохраняем в Valkey
    save_dataframe_to_valkey(top_products, "top_products", "product_id")
    
    # Общая выручка по категориям
    category_revenue = fact.join(dim_product, "product_id") \
        .groupBy("category") \
        .agg(
            sum("sale_total_price").alias("category_revenue"),
            count("sale_id").alias("sales_count")
        ) \
        .orderBy(desc("category_revenue"))
    
    print("Выручка по категориям:")
    category_revenue.show()
    
    # Сохраняем категории (создаем искусственный ID)
    category_revenue_with_id = category_revenue.withColumn(
        "category_id", 
        md5(col("category"))
    )
    save_dataframe_to_valkey(category_revenue_with_id, "category_revenue", "category_id")
    
    print("✅ Витрина продаж по продуктам успешно создана!")

def create_customer_sales_dashboard(fact, dim_customer):
    """
    Витрина 2: Продажи по клиентам
    """
    print("=== Создание витрины продаж по клиентам ===")
    
    # Топ-10 клиентов
    top_customers = fact.join(dim_customer, "customer_id") \
        .groupBy("customer_id", "first_name", "last_name", "country") \
        .agg(
            sum("sale_total_price").alias("total_spent"),
            count("sale_id").alias("total_orders"),
            avg("sale_total_price").alias("avg_order_value")
        ) \
        .orderBy(desc("total_spent")) \
        .limit(10)
    
    print("Топ-10 клиентов:")
    top_customers.show()
    
    save_dataframe_to_valkey(top_customers, "top_customers", "customer_id")
    
    # Распределение по странам
    country_distribution = fact.join(dim_customer, "customer_id") \
        .groupBy("country") \
        .agg(count("customer_id").alias("customer_count")) \
        .orderBy(desc("customer_count"))
    
    country_distribution_with_id = country_distribution.withColumn(
        "country_id", 
        md5(col("country"))
    )
    save_dataframe_to_valkey(country_distribution_with_id, "customers_by_country", "country_id")
    
    print("✅ Витрина продаж по клиентам успешно создана!")

def create_time_sales_dashboard(fact, dim_date):
    """
    Витрина 3: Продажи по времени
    """
    print("=== Создание витрины продаж по времени ===")
    
    # Месячные тренды
    monthly_trends = fact.join(dim_date, "date_id") \
        .groupBy("year", "month") \
        .agg(
            sum("sale_total_price").alias("monthly_revenue"),
            count("sale_id").alias("monthly_orders"),
            avg("sale_total_price").alias("avg_order_size")
        ) \
        .orderBy("year", "month")
    
    monthly_trends = monthly_trends.withColumn(
        "period_id", 
        concat(col("year"), lit("-"), lpad(col("month"), 2, "0"))
    )
    
    print("Месячные тренды:")
    monthly_trends.show()
    
    save_dataframe_to_valkey(monthly_trends, "monthly_sales", "period_id")
    print("✅ Витрина продаж по времени успешно создана!")

def create_store_sales_dashboard(fact, dim_store):
    """
    Витрина 4: Продажи по магазинам
    """
    print("=== Создание витрины продаж по магазинам ===")
    
    # Топ-5 магазинов
    top_stores = fact.join(dim_store, "store_id") \
        .groupBy("store_id", "store_name", "store_city", "store_country") \
        .agg(
            sum("sale_total_price").alias("total_revenue"),
            count("sale_id").alias("total_orders"),
            avg("sale_total_price").alias("avg_order_value")
        ) \
        .orderBy(desc("total_revenue")) \
        .limit(5)
    
    print("Топ-5 магазинов:")
    top_stores.show()
    
    save_dataframe_to_valkey(top_stores, "top_stores", "store_id")
    print("✅ Витрина продаж по магазинам успешно создана!")

def create_supplier_sales_dashboard(fact, dim_supplier, dim_product):
    """
    Витрина 5: Продажи по поставщикам
    """
    print("=== Создание витрины продаж по поставщикам ===")
    
    # Топ-5 поставщиков
    top_suppliers = fact.join(dim_supplier, "supplier_id") \
        .join(dim_product, "product_id") \
        .groupBy("supplier_id", "supplier_name", "supplier_country") \
        .agg(
            sum("sale_total_price").alias("total_revenue"),
            avg("price").alias("avg_product_price")
        ) \
        .orderBy(desc("total_revenue")) \
        .limit(5)
    
    print("Топ-5 поставщиков:")
    top_suppliers.show()
    
    save_dataframe_to_valkey(top_suppliers, "top_suppliers", "supplier_id")
    print("✅ Витрина продаж по поставщикам успешно создана!")

def create_product_quality_dashboard(dim_product, fact):
    """
    Витрина 6: Качество продукции
    """
    print("=== Создание витрины качества продукции ===")
    
    # Продукты с продажами и рейтингами
    product_ratings = dim_product \
        .join(fact.groupBy("product_id").agg(sum("sale_quantity").alias("total_sold")), "product_id", "left") \
        .select(
            "product_id", "name", "category", "rating", "reviews",
            coalesce("total_sold", lit(0)).alias("total_sold")
        ) \
        .filter(col("rating").isNotNull()) \
        .orderBy(desc("rating"))
    
    # Топ-10 по рейтингу
    top_rated = product_ratings.limit(10)
    print("Топ-10 продуктов по рейтингу:")
    top_rated.show()
    
    # Худшие по рейтингу
    worst_rated = product_ratings.orderBy("rating").limit(10)
    print("Худшие 10 продуктов по рейтингу:")
    worst_rated.show()
    
    save_dataframe_to_valkey(top_rated, "top_rated_products", "product_id")
    save_dataframe_to_valkey(worst_rated, "worst_rated_products", "product_id")
    
    print("✅ Витрина качества продукции успешно создана!")

def run_complete_valkey_etl():
    """Запуск полного ETL процесса в Valkey"""
    
    # Создаем Spark сессию
    spark = SparkSession.builder \
        .appName("ValkeyETL") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .getOrCreate()
    
    # Тестируем подключение
    if not test_valkey_connection():
        print("❌ Не удалось подключиться к Valkey. Проверьте конфигурацию.")
        spark.stop()
        return
    
    print("🚀 Начало ETL процесса в Valkey...")

        
    # Создаем все витрины
    create_product_sales_dashboard(fact, dim_product)
    create_customer_sales_dashboard(fact, dim_customer)
    create_time_sales_dashboard(fact, dim_date)
    create_store_sales_dashboard(fact, dim_store)
    create_supplier_sales_dashboard(fact, dim_supplier, dim_product)
    create_product_quality_dashboard(dim_product, fact)
    
    print("🎉 Все витрины успешно созданы в Valkey!")
        

# Запуск
run_complete_valkey_etl()

✓ Valkey подключение успешно
🚀 Начало ETL процесса в Valkey...
=== Создание витрины продаж по продуктам ===
Топ-10 продуктов:
+----------+---------+--------+--------------+-------------+----------+-------------+
|product_id|     name|category|total_quantity|total_revenue|avg_rating|total_reviews|
+----------+---------+--------+--------------+-------------+----------+-------------+
|       380|  Cat Toy|    Food|           408|     12538.44|   2.76667|        34470|
|        46| Dog Food|     Toy|           366|     15406.26|   2.98333|        22460|
|       995| Dog Food|    Cage|           348|     14523.84|   3.68333|        33010|
|       187|  Cat Toy|    Cage|           345|     11105.05|   2.28000|        28460|
|       673|Bird Cage|    Food|           320|      8453.16|   2.70000|        17940|
|       692|  Cat Toy|    Food|           320|     11856.56|   2.37500|        24670|
|       235|Bird Cage|    Food|           315|     12946.25|   3.32000|        26470|
|       699|Bi

25/11/19 09:55:42 ERROR Executor: Exception in task 0.0 in stage 12.0 (TID 8)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 840, in func
    return f(iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 1795, in func
    r = f(it)
  File "/tmp/ipykernel_15/1701722597.py", line 9, in save_partition
NameError: name 'VALKEY_CO

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 8) (98c2f2936152 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import json

# Конфигурация Valkey
VALKEY_CONFIG = {
    "host": "valkey",
    "port": 6379,
    "db": 0,
    "password": "bigdata2024",
    "decode_responses": True
}

def test_valkey_connection():
    """Тестирование подключения к Valkey"""
    try:
        import redis
        r = redis.Redis(**VALKEY_CONFIG)
        r.ping()
        print("✓ Valkey подключение успешно")
        return True
    except Exception as e:
        print(f"✗ Ошибка подключения к Valkey: {e}")
        return False

def save_dataframe_to_valkey(df, key_prefix, id_column="id"):
    """
    Сохраняет DataFrame в Valkey используя RDD
    """
    # Создаем broadcast переменную с конфигурацией
    config_broadcast = spark.sparkContext.broadcast(VALKEY_CONFIG)
    
    def save_partition(partition):
        import redis
        
        # Получаем конфигурацию из broadcast переменной
        config = config_broadcast.value
        
        # Создаем подключение для каждой партиции
        r = redis.Redis(**config)
        
        saved_count = 0
        for row in partition:
            try:
                # Создаем ключ
                key = f"{key_prefix}:{row[id_column]}"
                
                # Сохраняем все поля как hash
                row_dict = row.asDict()
                for field, value in row_dict.items():
                    if value is not None:
                        r.hset(key, field, str(value))
                
                saved_count += 1
                
            except Exception as e:
                print(f"Ошибка при сохранении {key}: {e}")
        
        r.close()
        print(f"Сохранено записей в партиции: {saved_count}")
    
    # Сохраняем через RDD
    df.rdd.foreachPartition(save_partition)
    print(f"✅ Успешно сохранено в Valkey с префиксом: {key_prefix}")

def create_product_sales_dashboard(fact, dim_product):
    """
    Витрина 1: Продажи по продуктам
    """
    print("=== Создание витрины продаж по продуктам ===")
    
    # Топ-10 самых продаваемых продуктов
    top_products = fact.join(dim_product, "product_id") \
        .groupBy("product_id", "name", "category") \
        .agg(
            sum("sale_quantity").alias("total_quantity"),
            sum("sale_total_price").alias("total_revenue"),
            avg("rating").alias("avg_rating"),
            sum("reviews").alias("total_reviews")
        ) \
        .orderBy(desc("total_quantity")) \
        .limit(10)
    
    print("Топ-10 продуктов:")
    top_products.show()
    
    # Сохраняем в Valkey
    save_dataframe_to_valkey(top_products, "top_products", "product_id")
    
    # Общая выручка по категориям
    category_revenue = fact.join(dim_product, "product_id") \
        .groupBy("category") \
        .agg(
            sum("sale_total_price").alias("category_revenue"),
            count("sale_id").alias("sales_count")
        ) \
        .orderBy(desc("category_revenue"))
    
    print("Выручка по категориям:")
    category_revenue.show()
    
    # Сохраняем категории (создаем искусственный ID)
    category_revenue_with_id = category_revenue.withColumn(
        "category_id", 
        md5(col("category"))
    )
    save_dataframe_to_valkey(category_revenue_with_id, "category_revenue", "category_id")
    
    print("✅ Витрина продаж по продуктам успешно создана!")

def create_customer_sales_dashboard(fact, dim_customer):
    """
    Витрина 2: Продажи по клиентам
    """
    print("=== Создание витрины продаж по клиентам ===")
    
    # Топ-10 клиентов
    top_customers = fact.join(dim_customer, "customer_id") \
        .groupBy("customer_id", "first_name", "last_name", "country") \
        .agg(
            sum("sale_total_price").alias("total_spent"),
            count("sale_id").alias("total_orders"),
            avg("sale_total_price").alias("avg_order_value")
        ) \
        .orderBy(desc("total_spent")) \
        .limit(10)
    
    print("Топ-10 клиентов:")
    top_customers.show()
    
    save_dataframe_to_valkey(top_customers, "top_customers", "customer_id")
    
    # Распределение по странам
    country_distribution = fact.join(dim_customer, "customer_id") \
        .groupBy("country") \
        .agg(count("customer_id").alias("customer_count")) \
        .orderBy(desc("customer_count"))
    
    country_distribution_with_id = country_distribution.withColumn(
        "country_id", 
        md5(col("country"))
    )
    save_dataframe_to_valkey(country_distribution_with_id, "customers_by_country", "country_id")
    
    print("✅ Витрина продаж по клиентам успешно создана!")

def create_time_sales_dashboard(fact, dim_date):
    """
    Витрина 3: Продажи по времени
    """
    print("=== Создание витрины продаж по времени ===")
    
    # Месячные тренды
    monthly_trends = fact.join(dim_date, "date_id") \
        .groupBy("year", "month") \
        .agg(
            sum("sale_total_price").alias("monthly_revenue"),
            count("sale_id").alias("monthly_orders"),
            avg("sale_total_price").alias("avg_order_size")
        ) \
        .orderBy("year", "month")
    
    monthly_trends = monthly_trends.withColumn(
        "period_id", 
        concat(col("year"), lit("-"), lpad(col("month"), 2, "0"))
    )
    
    print("Месячные тренды:")
    monthly_trends.show()
    
    save_dataframe_to_valkey(monthly_trends, "monthly_sales", "period_id")
    print("✅ Витрина продаж по времени успешно создана!")

def create_store_sales_dashboard(fact, dim_store):
    """
    Витрина 4: Продажи по магазинам
    """
    print("=== Создание витрины продаж по магазинам ===")
    
    # Топ-5 магазинов
    top_stores = fact.join(dim_store, "store_id") \
        .groupBy("store_id", "store_name", "store_city", "store_country") \
        .agg(
            sum("sale_total_price").alias("total_revenue"),
            count("sale_id").alias("total_orders"),
            avg("sale_total_price").alias("avg_order_value")
        ) \
        .orderBy(desc("total_revenue")) \
        .limit(5)
    
    print("Топ-5 магазинов:")
    top_stores.show()
    
    save_dataframe_to_valkey(top_stores, "top_stores", "store_id")
    print("✅ Витрина продаж по магазинам успешно создана!")

def create_supplier_sales_dashboard(fact, dim_supplier, dim_product):
    """
    Витрина 5: Продажи по поставщикам
    """
    print("=== Создание витрины продаж по поставщикам ===")
    
    # Топ-5 поставщиков
    top_suppliers = fact.join(dim_supplier, "supplier_id") \
        .join(dim_product, "product_id") \
        .groupBy("supplier_id", "supplier_name", "supplier_country") \
        .agg(
            sum("sale_total_price").alias("total_revenue"),
            avg("price").alias("avg_product_price")
        ) \
        .orderBy(desc("total_revenue")) \
        .limit(5)
    
    print("Топ-5 поставщиков:")
    top_suppliers.show()
    
    save_dataframe_to_valkey(top_suppliers, "top_suppliers", "supplier_id")
    print("✅ Витрина продаж по поставщикам успешно создана!")

def create_product_quality_dashboard(dim_product, fact):
    """
    Витрина 6: Качество продукции
    """
    print("=== Создание витрины качества продукции ===")
    
    # Продукты с продажами и рейтингами
    product_ratings = dim_product \
        .join(fact.groupBy("product_id").agg(sum("sale_quantity").alias("total_sold")), "product_id", "left") \
        .select(
            "product_id", "name", "category", "rating", "reviews",
            coalesce("total_sold", lit(0)).alias("total_sold")
        ) \
        .filter(col("rating").isNotNull()) \
        .orderBy(desc("rating"))
    
    # Топ-10 по рейтингу
    top_rated = product_ratings.limit(10)
    print("Топ-10 продуктов по рейтингу:")
    top_rated.show()
    
    # Худшие по рейтингу
    worst_rated = product_ratings.orderBy("rating").limit(10)
    print("Худшие 10 продуктов по рейтингу:")
    worst_rated.show()
    
    save_dataframe_to_valkey(top_rated, "top_rated_products", "product_id")
    save_dataframe_to_valkey(worst_rated, "worst_rated_products", "product_id")
    
    print("✅ Витрина качества продукции успешно создана!")

def run_complete_valkey_etl():
    """Запуск полного ETL процесса в Valkey"""
    
    global spark
    
    # Создаем Spark сессию
    spark = SparkSession.builder \
        .appName("ValkeyETL") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .getOrCreate()
    
    # Тестируем подключение
    if not test_valkey_connection():
        print("❌ Не удалось подключиться к Valkey. Проверьте конфигурацию.")
        spark.stop()
        return
    
    print("🚀 Начало ETL процесса в Valkey...")
    
    # Конфигурация PostgreSQL
    pg_url = "jdbc:postgresql://postgres:5432/bigdata"
    pg_properties = {
        "user": "postgres",
        "password": "postgres", 
        "driver": "org.postgresql.Driver"
    }
    
    try:
        # Загружаем данные из PostgreSQL
        print("📥 Загрузка данных из PostgreSQL...")
        
        fact = spark.read.jdbc(url=pg_url, table="fact_sales", properties=pg_properties)
        dim_product = spark.read.jdbc(url=pg_url, table="dim_product", properties=pg_properties)
        dim_customer = spark.read.jdbc(url=pg_url, table="dim_customer", properties=pg_properties)
        dim_store = spark.read.jdbc(url=pg_url, table="dim_store", properties=pg_properties)
        dim_supplier = spark.read.jdbc(url=pg_url, table="dim_supplier", properties=pg_properties)
        dim_date = spark.read.jdbc(url=pg_url, table="dim_date", properties=pg_properties)
        
        print("✅ Данные успешно загружены из PostgreSQL")
        
        # Показываем схемы данных для отладки
        print("Схема fact_sales:")
        fact.printSchema()
        
        print("Схема dim_product:")
        dim_product.printSchema()
        
        # Создаем все витрины
        create_product_sales_dashboard(fact, dim_product)
        create_customer_sales_dashboard(fact, dim_customer)
        create_time_sales_dashboard(fact, dim_date)
        create_store_sales_dashboard(fact, dim_store)
        create_supplier_sales_dashboard(fact, dim_supplier, dim_product)
        create_product_quality_dashboard(dim_product, fact)
        
        print("🎉 Все витрины успешно созданы в Valkey!")
        
    except Exception as e:
        print(f"❌ Ошибка в ETL процессе: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        spark.stop()
        print("🛑 Spark сессия остановлена")

# Запуск
if __name__ == "__main__":
    run_complete_valkey_etl()

✓ Valkey подключение успешно
🚀 Начало ETL процесса в Valkey...
📥 Загрузка данных из PostgreSQL...
❌ Ошибка в ETL процессе: An error occurred while calling o162.jdbc.
: org.postgresql.util.PSQLException: FATAL: password authentication failed for user "postgres"
	at org.postgresql.core.v3.ConnectionFactoryImpl.doAuthentication(ConnectionFactoryImpl.java:693)
	at org.postgresql.core.v3.ConnectionFactoryImpl.tryConnect(ConnectionFactoryImpl.java:203)
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:258)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:54)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:263)
	at org.postgresql.Driver.makeConnection(Driver.java:443)
	at org.postgresql.Driver.connect(Driver.java:297)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.co

Traceback (most recent call last):
  File "/tmp/ipykernel_15/905344476.py", line 283, in run_complete_valkey_etl
    fact = spark.read.jdbc(url=pg_url, table="fact_sales", properties=pg_properties)
  File "/opt/spark/python/pyspark/sql/readwriter.py", line 946, in jdbc
    return self._df(self._jreader.jdbc(url, table, jprop))
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/opt/spark/python/pyspark/errors/exceptions/captured.py", line 179, in deco
    return f(*a, **kw)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: An error occurred while calling o162.jdbc.
: org.postgresql.util.PSQLException: FATAL: password authentication failed for user "postgres"
	at org.postgresql.core.v3.ConnectionFactoryImpl.doAuthentication(ConnectionFactoryImpl.java:693)
	at org.postgresql.core.v3.ConnectionF

🛑 Spark сессия остановлена
